# Import section

In [1]:
from stage2 import *
import numpy as np
from astropy.io.votable import parse

# Creating a general class to gather group information into one object

In [2]:
class groupStructure:
    
    def __init__(self, groupName, listGals, redshifts, lsf, 
                 wavelengthOfObservation=3729, PA=0, inclination=-30, kinematicalPA=0,
                 XcenPos=15.0, YcenPos=15.0, RA=np.nan, DEC=np.nan,
                 lmass=np.nan, lmassLE=np.nan, lmassUE=np.nan,
                 lsfr=np.nan, lsfrLE=np.nan, lsfrUE=np.nan, mag=np.nan):
        '''
        Intialisation of the group structure. Parameters should be given with the right units. See mandatory and optional inputs for more information.
        
        Mandatory inputs
        ----------------
        groupName : string
            the name of the group
        listGals : list of strings
            the path to the gals folders
        lsf : list of floats
            the line spread function FWHM of the galaxies in km/s    
        redshifts : list of floats
            the redshifts of the galaxies
            
        Optional inputs
        ---------------
        DEC : list of floats
            the declination of the galaxy centre (degrees only)
        inclination : list of floats
            the inclination of the galaxies on the sky (between 0° and +90°)
        kinematicalPA : list of floats
            the kinematical PA as returned from a kinematical model
        lmass : list of floats
            log10 of the mass of the galaxies in solar masses
        lmassLE : list of floats
            lower error (1 sigma) on the log10 of the mass
        lmassUE : list of floats
            upper error (1 sigma) on the log10 of the mass
        lsfr : list of floats
            log10 of the SFR of the galaxies in solar masses per year
        lsfrLE : list of floats
            lower error (1 sigma) on the log10 of the sfr
        lsfrUE : list of floats
            upper error (1 sigma) on the log10 of the sfr
        mag : list of floats
            the magnitude (apparent or absolute)of the galaxies
        PA : list of floats
            the morphological position angle of the galaxies (between -90° and +90°)
        RA : list of floats
            the right ascension of the galaxy centre (degrees only)
        wavelengthOfObservation : list of integers/floats
            the wavelength at which we observed the galaxies (default is OII, must be given in Angstroms)
        XcenPos : list of floats
            the X position in pixel of the centre in a MUSE image of 30x30px
        YcenPos : list of floats
            the Y position in pixel of centre in a MUSE image of 30x30px
            
        Attributes
        ----------
            
            Attributes units
            ----------------
            To each attribute corresponds a unit marked with two _ before the attribute name.
            Some of these are used to convert values from one unit to another or to print them in a certain format. Do not change them directly but use the relevant functions.
            Avaiblable units are
                angles     : deg, rad, arcmin, arcsec, hours, minutes, seconds
                length     : px
                wavelength : angstrom, nm
                
        __hstGroup : str
            an identifier for the group to find hst images
        __ln : int
            the number of galaxies associated to the object. It is computed at the creation of the object and should not be changed after.
        dec : list of floats
            declination (degrees only) of each galaxy
        group : str
            the group name
        galsFolder : str
            the group folder where the galaxies folders are stored
        galsNames : list of str
            the name of the galaxies
        galsNum : list of int
            the number associated to each galaxy
        hstFolder : str
            the group folder where the hst images of the galaxies are stored
        inclination : list of int
            the inclination of the galaxies on the sky. 
        kin_PA : list of floats
            the kinematical position angle of each galaxy (between -90° and +90°). Default is 0°.
        lsf : list of floats
            the line spread function of each galaxy
        lsfr : list of floats
            log10 of the SFR of the galaxies
        lsfr_le : list of floats
            lower error (1 sigma) on the log10 of the sfr
        lsfr_ue : list of floats
            upper error (1 sigma) on the log10 of the sfr
        mag : list of floats
            the apparent or absolute magnitude of the galaxies
        PA : list of floats
            the morphological position angle of each galaxy (between -90° and +90°). If no value or a None is given, default will be put to -30°.
        psf : list of floats
            the psf at the redshift of every galaxy in pixel. This requires to know the redshift beforehand.
        ra : list of floats
            right ascension (degrees only) of each galaxy
        wv : list of floats
            the wavelength of observation (redshifted) in Angstrom of the galaxies. This requires to know the redshift beforehand.
        XcenPos : list of floats
            the X position in pixel of the centre in a MUSE image of 30x30px of every galaxy
        YcenPos : list of floats
            the Y position in pixel of centre in a MUSE image of 30x30px of every galaxy
        z : list of floats
            the resdhift of each galaxy
        '''

        ##Filling lists of mandatory fields
        #galaxies folders and names
        self.__ln          = len(listGals)
        self.group         = groupName
        self.__hstGroup    = self.group if self.group == "84-N" else  self.group.split('_')[0].split('-')[0]
        self.galsNames     = [gal.split("o2/")[1] for gal in listGals]
        self.galsFolder    = listGals[0].split("o2")[0]
        self.galsNum       = [int(gal.split("_")[-2]) for gal in self.galsNames]
        self.hstFolder     = ["/home/wilfried/ST2/data/hst/CGr" + self.__hstGroup + "/" + str(gnum) + "_CGr" + self.__hstGroup + ".fits" for gnum in self.galsNum]
        
        #redshift
        self.z             = [z for z in redshifts]
        
        #lsf FWHM
        self.lsf            = [l for l in lsf]
        self.__lsf_unit     = "km/s"
        
        ## Filling lists of optional fields
        #centre position
        try:
            len(XcenPos)
        except TypeError:
            XcenPos         = [XcenPos]*self.__ln
        self.XcenPos        = [np.nan if xcen is None else xcen for xcen in XcenPos]
        self.__XcenPos_unit = "px"
        
        try:
            len(YcenPos)
        except TypeError:
            YcenPos         = [YcenPos]*self.__ln
        self.YcenPos        = [np.nan if ycen is None else ycen for ycen in YcenPos]
        self.__YcenPos_unit = "px"
        
        try:
            len(RA)
        except TypeError:
            RA              = [RA]*self.__ln
        self.ra             = [np.nan if ra is None else ra for ra in RA]
        self.__ra_unit      = "deg"
        
        try:
            len(DEC)
        except TypeError:
            DEC             = [DEC]*self.__ln
        self.dec            = [np.nan if dec is None else dec for dec in DEC] 
        self.__dec_unit     = "deg"
        
        #define default wavelength of observation for this group (OII at the group redshift in Angstrom)
        try:
            len(wavelengthOfObservation)
        except TypeError:
            wavelengthOfObservation = [wavelengthOfObservation]*self.__ln
        self.wv             = [np.nan if wv is None else wv for wv in wavelengthOfObservation]
        self.wv             = [wv*(1+z) for wv, z in zip(self.wv, self.z)]
        self.__wv_unit      = "angstrom"
        
        #generate PSF FWHM for each galaxy
        self.psf            = computeGroupFWHM(self.wv, [self.group]*self.__ln, verbose=False, model='Gaussian')
        self.__psf_unit     = "px"
        
        #set default morphological PA
        try:
            len(PA)
        except TypeError:
            PA              = [PA]*self.__ln
        self.PA             = [np.nan if pa is None else pa for pa in PA]
        self.__PA_unit      = "deg"
        
        #set default kinematical PA
        try:
            len(kinematicalPA)
        except TypeError:
            kinematicalPA   = [kinematicalPA]*self.__ln
        self.kin_PA         = [np.nan is kpa if None else kpa for kpa in kinematicalPA]
        self.__kin_PA_unit  =  "deg"
        
        #set PAs between -90° and +90°
        
        #set default inclination
        try:
            len(inclination)
        except TypeError:
            inclination         = [inclination]*self.__ln
        self.inclination        = [np.nan if i is None else i for i in inclination]
        self.__inclination_unit = "sdeg"
        
        #set default mass and its errors
        try:
            len(lmass)
        except TypeError:
            lmass         = [lmass]*self.__ln
        self.lmass        = [np.nan if lm is None else lm for lm in lmass]
        self.__lmass_unit = "log10(M_sol)"
        
        try:
            len(lmassLE)
        except TypeError:
            lmassLE       = [lmassLE]*self.__ln
        self.lmassLE      = [np.nan if lmle is None else lmle for lmle in lmassLE]
        self.lmassLE_unit = "log10(M_sol)"
        
        try:
            len(lmassUE)
        except TypeError:
            lmassUE       = [lmassUE]*self.__ln
        self.lmassUE      = [np.nan if lmue is None else lmue for lmue in lmassUE]
        self.lmassUE_unit = "log10(M_sol)"
        
        #set default sfr and its errors
        try:
            len(lsfr)
        except TypeError:
            lsfr          = [lsfr]*self.__ln
        self.lsfr         = [np.nan if ls is None else ls for ls in lsfr]
        self.__lsfr_unit  = "log10(M_sol/yr)"
        
        try:
            len(lsfrLE)
        except TypeError:
            lsfrLE        = [lsfrLE]*self.__ln
        self.lsfrLE       = [np.nan if lsle is None else lsle for lsle in lsfrLE]
        self.lsfrLE_unit  = "log10(M_sol/yr)"
        
        try:
            len(lsfrUE)
        except TypeError:
            lsfrUE        = [lsfrUE]*self.__ln
        self.lsfrUE       = [np.nan if lsue is None else lsue for lsue in lsfrUE]
        self.lsfrUE_unit  = "log10(M_sol/yr)"
        
        #set default mag
        try:
            len(mag)
        except TypeError:
            mag           = [mag]*self.__ln
        self.mag          = [np.nan if mg is None else mg for mg in mag]
        self.__mag_unit   = "mag(L_sol)"
        
    def updateCenPos(self, Xpos, Ypos):
        '''
        Update galaxies centre X and Y position (in pixels in the MUSE image)
        
        Input
        -----
        Xpos : list of floats
            the X position of the galaxies centres
        Ypos : list of floats
            the Y position of the galaxies centres
        '''
        
        if (len(Xpos) != len(Ypos)) or (len(Xpos) != self.__ln):
            raise Exception("Given X and Y positions have length %d and %d but data has length %d." %(len(Xpos), len(Ypos), self.__ln))
        self.XcenPos = [i for i in Xpos]
        self.YcenPos = [i for i in Ypos]
        
    def updatePSF(self, wavelength, verbose=False):
        '''
        Update the PSF for each galaxy using the given wavelength of observation (not rest-frame)
        
        Input
        -----
        verbose : boolean
            whether to print the computed value on screen or not
        wavelength : list of integers
            the wavelength of observation equals to (1+z) times the rest-frame wavelength
        '''
        
        if len(wavelength) != self.__ln:
            raise Exception("Given wavelength list has length %d but data has length %d" %(len(wavelength), self.__ln))
        self.psf = computeGroupFWHM(wavelength, [self.group]*self.__ln, verbose=verbose)
        
    def updatePA(self, PA, unit="deg"):
        '''
        Update the PA of the galaxies
        
        Mandatory inputs
        ----------------
        PA : list of inetgers/floats
            the PA of the galaxies between -90° and +90°
            
        Optional inputs
        ---------------
        unit : string
            either deg or rad to indicate which unit it corresponds to
        '''
        
        #checking unit
        if unit in ('deg', 'degree', 'degrees'):
            unit = 'deg'
        elif unit in ('rad', 'radian', 'radians'):
            unit = 'rad'
        else:
            raise Exception("Unit %s was not recognised. Available units are %s." %(unit, ('deg', 'rad'))) 
        
        if len(PA) != self.__ln:
            raise Exception("Given PA list has length %d but data has length %d" %(len(PA), self.__ln))
        self.PA        = PA
        self.__PA_unit = unit
    
    def updateInclination(self, inclination, unit="deg"):
        '''
         Update the inclination of the galaxies
        
        Mandatory inputs
        ----------------
        inclination : list of floats
            the inclination of the galaxies (between the line of sight and the normal of the galactic plane) between 0° and 90°
            
        Optional inputs
        ---------------
        unit : string
            either deg or rad to indicate which unit it corresponds to
        '''
       
        #checking unit
        if unit in ('deg', 'degree', 'degrees'):
            unit = 'deg'
        elif unit in ('rad', 'radian', 'radians'):
            unit = 'rad'
        else:
            raise Exception("Unit %s was not recognised. Available units are %s." %(unit, ('deg', 'rad')))
        
        if len(inclination) != self.__ln:
            raise Exception("Given inclination list has length %d but data has length %d" %(len(inclination), self.__ln))
        self.inclination = inclination
        self.__inc_unit  = unit
        
    def updateKinPA(self, kinPA, unit="deg"):
        '''
        Update the kinematical PA of galaxies
        
        Mandatory inputs
        ----------------
        kinPA : list of floats
            the kinematical PA of galaxie given by a model
        
        Optional inputs
        ---------------
        unit : string
            either deg or rad to indicate which unit it corresponds to
        '''
        
        #checking unit
        if unit in ('deg', 'degree', 'degrees'):
            unit = 'deg'
        elif unit in ('rad', 'radian', 'radians'):
            unit = 'rad'
        else:
            raise Exception("Unit %s was not recognised. Available units are %s." %(unit, ('deg', 'rad')))
            
        if len(kinPA) != self.__ln:
            raise Exception("Given kinematical PA list has length %d but data has length %d" %(len(kinPA), self.__ln))
        self.kin_PA         = kinPA
        self.__kin_PA_unit  = unit
        
        
    def changePAunit(self, newUnit, morpho=True, kinematical=True):
        '''
        Change the values and the unit of the morphological and kinematical PAs
        
        Mandatory inputs
        ---------------
        newUnit : str
            A string indicating the new unit for the PA. Either deg for degree or rad for radian.
            
        Optional inputs
        ---------------
        morpho : boolean
            Whether to apply the change of unit to the morphological PA
        kinematical : boolean
            Whether to apply the change of unit to the kinematical PA

        Return a tuple with either the modified morphological/kinematical PA or both
        '''
        
        def convertDegtoToRad(dataIn):
            return [dt*np.pi/180.0 for dt in dataIn]
        
        def convertRadToDeg(dataIn):
            return [dt*180.0/np.pi for dt in dataIn]
        
        if (not morpho) and (not kinematical):
            raise Exception("At least the morphological or kinematical PAs must be chosen for the change of unit. None was selected.")
            
        if newUnit in ('deg', 'degree', 'degrees'):
            newUnit = 'deg'
        elif newUnit in ('rad', 'radian', 'radians'):
            newUnit = 'rad'
        else:
            raise Exception("Unit %s was not recognised. Available units are %s." %(newUnit, ('deg', 'rad')))
            
        #changing unit
        if morpho:
            if (newUnit == 'deg') and (self.__PA_unit == 'rad'):
                self.PA                = convertRadToDeg(self.PA)
                self.__PA_unit         = newUnit
            elif (newUnit == 'rad') and (self.__PA_unit == 'deg'):
                self.PA                = convertDegtoToRad(self.PA)
                self.__PA_unit         = newUnit
        if kinematical:
            if (newUnit == 'deg') and (self.__kin_PA_unit == 'rad'):
                self.kin_PA            = convertRadToDeg(self.kin_PA)
                self.__kin_PA_unit     = newUnit
            elif (newUnit == 'rad') and (self.__kin_PA_unit == 'deg'):
                self.kin_PA            = convertDegtoToRad(self.kin_PA)
                self.__kin_PA_unit     = newUnit
                
        if morpho and kinematical:
            return self.PA, self.kin_PA
        elif morpho:
            return self.PA
        else:
            return self.kin_PA

# Getting list of galaxies

In [3]:
#input file
galsLocation = "SelectedGals_outputFolders"
galsZfile    = "SelectedGals"
galsLSFfile  = "SelectedGals_clean_o2"

#get gals location
listGals     = np.genfromtxt(galsLocation, dtype="str")

#keep folders whereto the input file will be written
galsFolders  = [gal.split("o2")[0] for gal in listGals]
#keep gal name for the input file
galNames     = [gal.split("o2/")[1] for gal in listGals]

#get gals redshift
galsZ        = np.genfromtxt(galsZfile, dtype='float', usecols=1)
print("Same size ?", len(galsZ)==len(galNames))

#get gals lsf FWHM (in km/s)
galsLSF      = np.genfromtxt(galsLSFfile, dtype='float', usecols=1)
print("Same size ?", len(galsLSF)==len(galNames))

Same size ? True
Same size ? True


# Recover PA (for some galaxies) and b/a values from the .vot file

## Get full data from the .vot file

In [4]:
votFile = "../outputs/selected_field_gals.vot"

#checking that it is a vot file and retrieve data
if is_VOtable(votFile):
    full = parse(votFile).get_first_table().array
    
#Get gals number
galsNum = [int(num.split('_')[-2]) for num in listGals]

#Get gals group and convert it into the same format as that in the .vot catalogue
groupList = [name.split('MUSE/')[1].split('/o2')[0].split('_s')[0].split('CGr')[1] for name in listGals]
for pos, gr in enumerate(groupList):
    if gr == "84-N":
        groupList[pos] = 84.1
    elif gr in ("84_d", "34_d", "79_d", "30_d", "84_bs", "34_bs", "79_bs", "30_bs"):
        groupList[pos] = float(gr.split("_")[0])
    elif gr in ('32-M1_d', '32-M2_d', '32-M3_d'):
        groupList[pos] = 32.0
    else:
        groupList[pos] = float(gr)

The file ../outputs/selected_field_gals.vot is a VOtable, right ? True


## Gals in CGr84 have the wrong MUSE Id in their folder name so we need to associate them with the correct one using another file

In [5]:
matchingFile = '../data/catalogues/CGR84-83_FD+North_zcatalog_withLaigle+16_withFAST_withnewPLATEFIT_totalflux_withnewz_jan19_withFOF_withGALFIT_withGALKIN_jan19.vot'
if is_VOtable(matchingFile):
    tmpCOSMOSID   = parse(matchingFile).get_first_table().array['COSMOS_Group_Number']
    m             = tmpCOSMOSID==84.0
    #the corresponding MUSE id in the .vot file with the morphological information
    CatalogMUSEID = parse(matchingFile).get_first_table().array['ID'][m]
    #the MUSE id found in the folders
    FolderMUSEID  = parse(matchingFile).get_first_table().array['ID_in_MUSE_Field'][m]
else:
    raise Exception("The file", matchingFile, "could not be found")

newGalsNums = galsNum[:]
for pos in np.where(np.asarray(groupList)==84.0)[0]:
    newGalsNums[pos] = CatalogMUSEID[FolderMUSEID==galsNum[pos]][0]
print(np.asarray(newGalsNums) == np.asarray(galsNum))
print(newGalsNums)

The file ../data/catalogues/CGR84-83_FD+North_zcatalog_withLaigle+16_withFAST_withnewPLATEFIT_totalflux_withnewz_jan19_withFOF_withGALFIT_withGALKIN_jan19.vot is a VOtable, right ? True
[ True  True  True False False False False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True]
[8, 20, 21, 220, 235, 313, 324, 25, 25, 50, 50, 90, 90, 103, 103, 123, 123, 144, 144, 154, 154, 155, 155, 41, 42, 51, 80, 112

## Recover morphological information (PA, i, mag, RA, DEC, SFR, mass)

In [6]:
#ID and group number to match galaxies
fullGrNb    = full['COSMOS_Group_Number']
fullGalNb   = full['ID']
#complete morphological info from the catalogue
fullPA      = full['PA_GIM2D_ZURICH']
fullb_a     = full['Axial_Ratio_CASSATA']
fullRA      = full['RA']
fullDEC     = full['DEC']
fullMass    = full['MASS_MED']
fullMass_le = full['MASS_MED_MIN68']
fullMass_ue = full['MASS_MED_MAX68']
fullSFR     = full['SFR_MED']
fullSFR_le  = full['SFR_MED_MIN68']
fullSFR_ue  = full['SFR_MED_MAX68']
fullMag     = full['MAG_AUTO_ACS_CASSATA']

#we find the PA and the inclination for the desired galaxies by matching their group and galaxy numbers
PAlist      = []
incList     = []
RAlist      = []
DEClist     = []
MassList    = []
Mass_leList = []
Mass_ueList = []
SFRlist     = []
SFR_leList  = []
SFR_ueList  = []
magList     = []
for gal, gr in zip(newGalsNums, groupList):
    
    #we get the position in the full array where the galaxy lies
    pos = np.where(np.logical_and(fullGrNb == gr, fullGalNb == gal))[0]
    
    ## Writing PA
    #we get the corresponding PA
    tmp = fullPA[pos]
    #if the value is masked, i.e. missing, we put by default 60 degrees
    if tmp.mask[0] :
        tmp = 60.0
    else:
        tmp = tmp[0]
    #if the value is -9999.0 (galaxy found in Cassata catalogue but not in Zurich), we put by default 60 degrees
    if tmp == -9999.0:
        tmp = 60.0
    #changing PA definition from [0, 180]° to [-90, +90]°
    PAlist.append(tmp-90)
    
    ## Writing inclination
    #we get the corresponding b/a, compute the inclination (between 0° and 90°) from it and add it to the list
    tmp = fullb_a[pos][0]
    incList.append(np.arccos(tmp)*180/np.pi)
    
    ## Writing RA, DEC in degrees
    RAlist.append(fullRA[pos][0])
    DEClist.append(fullDEC[pos][0])
    
    ## Writing Mass
    MassList.append(fullMass[pos][0])
    Mass_leList.append(fullMass_le[pos][0])
    Mass_ueList.append(fullMass_ue[pos][0])
    
    ## Writing SFR
    SFRlist.append(fullSFR[pos][0])
    SFR_leList.append(fullSFR_le[pos][0])
    SFR_ueList.append(fullSFR_ue[pos][0])
    
    ## Writing magnitude
    magList.append(fullMag[pos][0])
    
#converting the lists into arrays
def returnAnArray(listofLists):
    return [np.asarray(i) for i in listofLists]

PAlist, incList, RAlist, DEClist, MassList, Mass_leList, Mass_ueList, SFRlist, SFR_leList, SFR_ueList, magList  = np.asarray([PAlist, incList, RAlist, DEClist, MassList, Mass_leList, 
                                                                                                                     Mass_ueList, SFRlist, SFR_leList, SFR_ueList, magList])

# Constructing the group objects

In [7]:
#first we create a dict with the indices of galaxies belonging to each group
names    = np.asarray([i.split("MUSE/")[1].split("/o2")[0].split("_s")[0] for i in listGals])
unames   = np.unique(names)
maskDict = {}
for gr in unames:
    maskDict.update({gr : np.where(names==gr)})
    
tmpNames   = [name.split('CGr')[1] for name in unames]
for pos, gr in enumerate(tmpNames):
    if gr in ("84_d", "79_d", "84_bs", "79_bs"):
        tmpNames[pos] = gr.split("_")[0]
    elif gr in ('32-M1_d', '32-M2_d', '32-M3_d'):
        tmpNames[pos] = gr.split('_')[0]

#then we create a dictionnary combining all the group objects
GroupsDict = {}
for gr, tmp in zip(unames, tmpNames):
    m = maskDict[gr][0]
    GroupsDict.update({gr : groupStructure(tmp, 
                                           listGals[m],
                                           galsZ[m],
                                           galsLSF[m],
                                           PA = PAlist[m], inclination = incList[m],
                                           RA = RAlist[m], DEC=DEClist[m],
                                           lmass = MassList[m], lmassLE = Mass_leList[m], lmassUE = Mass_ueList[m],
                                           lsfr = SFRlist[m], lsfrLE = SFR_leList[m], lsfrUE = SFR_ueList[m],
                                           mag = magList[m])})

# Generating the kinematic input file (run it once and then manually modify the values which seem inconsistent)

# Generating the maps_input file

## Getting PA and (X, Y) centre position from recap file and converting it to the relevant range

In [8]:
#GroupTest = {'CGr23' : GroupsDict['CGr84-N']}
GroupTest = GroupsDict
Groups    = GroupTest

In [112]:
for gr in Groups.values():
    
    file               = gr.galsFolder + "o2/recap_kinematics_parameters_2_slp_xyi_mclean5.0.txt"
    gal                = np.genfromtxt(file, usecols=(0), dtype='str', skip_header=1)
    kinPA, Xcen, Ycen  = np.genfromtxt(file, usecols=(7, 1, 3), dtype='float', skip_header=1, unpack=True)
    
    #re ordering the kinematical PA so that it corresponds to the correct galaxy
    kinPA_ordered = []
    gal_ordered   = []
    Xcen_ordered  = []
    Ycen_ordered  = []
    for name in gr.galsNames:
        kinPA_ordered.append(kinPA[gal==name][0])
        Xcen_ordered.append(Xcen[gal==name][0])
        Ycen_ordered.append(Ycen[gal==name][0])
        gal_ordered.append(gal[gal==name][0])
        
    gr.updateKinPA(kinPA_ordered)
    gr.updateCenPos(Xcen_ordered, Ycen_ordered)

## Writing data into file

In [113]:
for group in Groups.values():
    outputFolder = group.galsFolder
    outputFile   = "maps_input_o2.txt"
    
    #generate lists with default values we do not change
    sz           = len(group.galsNames)
    vs           = [0.0]*sz
    opt1         = ['mclean5.0']*sz
    line         = ['OII3729']*sz
    opt2         = ['xyi']*sz
    r12          = [1.0]*sz
    param1       = [0]*sz
    param2       = ['RDPRMS']*sz
    param3       = param1
    param4       = param1
    
    #generate header
    hdr          = " name ID morpho_image opt1   line   opt2    x        y        pa       i     vs  zpsf     z     r12   psf"
    
    #converting psf in arcsec
    psf          = [np.round(i[1]*0.2, 3) for i in group.psf]
    data         = [group.galsNames, [gr.split("_o2")[0] for gr in group.galsNames], group.hstFolder,
                    opt1, line, opt2, np.round(group.XcenPos, 2), np.round(group.YcenPos, 2),
                    np.round(group.kin_PA, 1), [int(gr) for gr in group.inclination],
                    vs, np.round(group.lsf, 2), np.round(group.z, 3), r12, psf,
                    param1, param2, param3, param4]
    
    data         = np.asarray(data).T
    np.savetxt(outputFolder+outputFile,
               data,
               header=hdr, comments="#", delimiter="   ",
               fmt="%s", )